In [1]:
import json
import os
from collections import defaultdict
import fiftyone as fo
import fiftyone.utils.random as four
from fiftyone import ViewField as F

In [6]:
def export_yolo_data(
    samples,
    export_dir,
    classes,
    label_field = "ground_truth",
    split = None
    ):

    if type(split) == list:
        splits = split
        for split in splits:
            export_yolo_data(
                samples,
                export_dir,
                classes,
                label_field,
                split
            )
    else:
        if split is None:
            split_view = samples
            split = "val"
        else:
            split_view = samples.match_tags(split)

        split_view.export(
            export_dir=export_dir,
            dataset_type=fo.types.YOLOv5Dataset,
            label_field=label_field,
            classes=classes,
            split=split
        )

In [14]:
def add_yolo_detections(
    samples,
    prediction_field,
    prediction_filepath,
    class_list
    ):

    prediction_filepaths = samples.values(prediction_filepath)
    yolo_detections = [read_yolo_detections_file(pf) for pf in prediction_filepaths]
    detections =  [convert_yolo_detections_to_fiftyone(yd, class_list) for yd in yolo_detections]
    samples.set_values(prediction_field, detections)

In [7]:
DATASET_ROOT = '/media/datasets/DATASET'
FRAMES_ROOT = os.path.join(DATASET_ROOT, 'FRAMES')
DATASET_CFG_PATH = 'metadata/set_utf.cfg'
IMG_WIDTH = 960
IMG_HEIGHT = 600

In [8]:
classes = [str(x) for x in range(14)]
classes

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13']

In [9]:
dataset_dict = defaultdict(list)

with open(os.path.join(DATASET_ROOT, DATASET_CFG_PATH), 'r') as fd:
    fname = ''
    for line in fd:
        # print(line)
        line = line.strip()
        if line.split('.')[-1] == 'frame':
            fname = os.path.join(FRAMES_ROOT, *line.split('\\'))
            fname = fname.replace('.frame', '.bmp')
            # print(fname)
            # print(os.path.isfile(os.path.join(FRAMES_ROOT, fname)))
        else:
            # fill objects info
            x_px, y_px, cls = list(map(int, line.replace(' ', '').split(',')))
            x = x_px / IMG_WIDTH
            y = y_px / IMG_HEIGHT
            dataset_dict[fname].append({'x': x, 'y': y, 'x_px': x_px, 'y_px': y_px, 'cls': cls})

In [10]:
dataset_dict['/media/datasets/DATASET/FRAMES/0/1538/frame0006.bmp']

[{'x': 0.8427083333333333, 'y': 0.56, 'x_px': 809, 'y_px': 336, 'cls': 3},
 {'x': 0.6260416666666667,
  'y': 0.49833333333333335,
  'x_px': 601,
  'y_px': 299,
  'cls': 0}]

In [11]:
dataset = fo.Dataset.from_images_dir(
    images_dir=FRAMES_ROOT,
    name='hackatom',
    overwrite=True
    )
dataset.persistent = True
session = fo.launch_app(dataset, address="0.0.0.0")

 100% |███████████████| 7808/7808 [1.2s elapsed, 0s remaining, 6.7K samples/s]         


In [12]:
preds_view = dataset.take(len(dataset))

In [13]:
with fo.ProgressBar() as pb:
    for sample in pb(preds_view):
        if sample.filepath in dataset_dict.keys():
            keypoints = []
            detections = []
            for kp in dataset_dict[sample.filepath]:
                keypoints.append(
                    fo.Keypoint(
                        label=str(kp['cls']),
                        points=[(kp['x'], kp['y'])],
                    )
                )
                x_tl_px = (kp['x_px'] - 80) / IMG_WIDTH
                y_tl_px = (kp['y_px'] - 80) / IMG_HEIGHT
                width = 160 / IMG_WIDTH
                height = 160 / IMG_HEIGHT
                detections.append(
                    fo.Detection(
                        label=str(kp['cls']),
                        bounding_box=[x_tl_px, y_tl_px, width, height]
                    )
                )
            # sample['keypoints'] = fo.Keypoints(keypoints=keypoints)
            sample['ground_truth'] = fo.Detections(detections=detections)
            sample.save()

   0% ||--------------|    0/7808 [35.7ms elapsed, ? remaining, ? samples/s] 

 100% |███████████████| 7808/7808 [18.7s elapsed, 0s remaining, 431.6 samples/s]      


In [11]:
len(dataset)

7808

In [14]:
non_empty = preds_view.match(
{
  "$and": [
    {
      "ground_truth.detections.label": {
        "$in": [
          "1",
          "10",
          "11",
          "12",
          "13",
          "2",
          "3",
          "4",
          "5",
          "6",
          "7",
          "8",
          "9"
        ]
      }
    }
  ]
}).shuffle()

In [15]:
len(non_empty)

4640

In [17]:
test_dataset = non_empty[:1200]
train_dataset = non_empty[1200:]

In [18]:
# ## split into train and val
# four.random_split(
#     train_dataset,
#     {"train": 0.8, "val": 0.2}
# )

# ## export in YOLO format
# export_yolo_data(
#     train_dataset,
#     "../data_train_balanced",
#     classes,
#     split = ["train", "val"]
# )

 100% |███████████████| 2752/2752 [15.2s elapsed, 0s remaining, 180.3 samples/s]      
Directory '../data_train_balanced' already exists; export will be merged with existing files
 100% |█████████████████| 688/688 [4.0s elapsed, 0s remaining, 181.3 samples/s]      


In [19]:
# ## export in YOLO format
# export_yolo_data(
#     test_dataset,
#     "../data_test_balanced",
#     classes
# )

 100% |███████████████| 1200/1200 [6.4s elapsed, 0s remaining, 192.3 samples/s]      


In [ ]:
#!yolo task=detect mode=train model=yolov8l.pt data=../data_train/dataset.yaml epochs=300 patience=30 imgsz=640 batch=192 device=0,1,2,3,4,5 workers=44 plots=True

In [ ]:
#!yolo task=detect mode=predict model=runs/detect/train14/weights/best.pt source=../data_test_balanced/images/val device=0,1,2,3,4,5 save_txt=True save_conf=True imgsz=608,960

In [ ]:
filepaths = test_dataset.values("runs/detect/predict3/labels")
prediction_filepaths = [get_prediction_filepath(fp, run_number=2) for fp in filepaths]

test_dataset.set_values(
    "yolov8m_640",
    prediction_filepaths
)

add_yolo_detections(
    test_dataset,
    "yolov8m_640",
    "yolov8n_bird_det_filepath",
    classes
)